In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.wh

### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. Pre-training σε μεγάλα unlabelled datasets. Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. Fine-tuning σε μικρότερα labelled datasets. Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του text-classification pipeline μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης. Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες [entailment/neutral/contradiction].

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι γραμματικά ορθό (acceptable) ή όχι (unacceptable):

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια. Τα reviews αυτά χωρίζονται σε κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.

In [ ]:
from datasets import load_dataset

# insert your code here
dataset = load_dataset('yelp_polarity')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [ ]:
# insert your code here

# Keep just 300 samples
train_dataset = dataset['train'].shuffle(seed=42).select(range(300))
test_dataset = dataset['test'].shuffle(seed=42).select(range(300))

In [ ]:
print("Train dataset", train_dataset)
print("Test dataset", test_dataset)

Train dataset Dataset({
    features: ['text', 'label'],
    num_rows: 300
})
Test dataset Dataset({
    features: ['text', 'label'],
    num_rows: 300
})


# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα. Η διαδικασία αυτή επιτελείται μέσω των Tokenizers, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer. Επιλέγοντας τη μέθοδο from_pretrained λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT το οποίο διδαχθήκατε στο εργαστήριο, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [ ]:
# insert your code here


from transformers import AutoTokenizer, DistilBertForSequenceClassification


# Load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


###MY COMMENT
Επιλέγουμε το μοντέλο distilbert for sequence classification γιατί τρέχει 60% πιο γρήγορα από το BERT, ενώ διατηρεί πάνω από το 95% της απόδοσης του BERT.

Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# insert your code here
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

In [ ]:
print("Tokenized train dataset:", tokenized_train_dataset)
print("Tokenized test dataset:", tokenized_test_dataset)

Tokenized train dataset: Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 300
})
Tokenized test dataset: Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 300
})


## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [4]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή.

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε? Σχολιάστε και αναλύστε.

### Πειραματισμοί με Υπερπαραμέτρους και Αποτελέσματα
###(I wrote this)
Ακολουθούν τα αποτελέσματα των πειραματισμών με διαφορετικές υπερπαραμέτρους για το fine-tuning του μοντέλου `DistilBertForSequenceClassification` στο dataset Yelp Polarity.

#### Πίνακας Αποτελεσμάτων

| Learning Rate | Batch Size | Epochs | Optimizer   | Scheduler | Accuracy |
|---------------|------------|--------|-------------|-----------|----------|
| 5e-5          | 8          | 3      | AdamW       | Linear    | 90.00%   |
| 5e-5          | 8          | 3      | AdamW       | Cosine    | 89.66%   |
| 5e-5          | 8          | 5      | AdamW       | Linear    | 89.33%   |
| 5e-5          | 8          | 3      | AdamW       | Cosine    | 89.33%   |
| 5e-5          | 16         | 3      | AdamW       | Linear    | 90.00%   |
| 5e-5          | 16         | 3      | AdamW       | Cosine    | 90.00%   |
| 5e-5          | 16         | 5      | AdamW       | Linear    | 89.67%   |
| 5e-5          | 16         | 5      | AdamW       | Cosine    | 91.00%   |
| 5e-5          | 32         | 3      | AdamW       | Linear    | 82.00%   |
| 5e-5          | 32         | 3      | AdamW       | Cosine    | 86.33%   |
| 5e-5          | 32         | 5      | AdamW       | Linear    | 91.67%   |
| 5e-5          | 32         | 5      | AdamW       | Cosine    | 89.67%   |
| 3e-5          | 8          | 3      | AdamW       | Linear    | 89.67%   |
| 3e-5          | 8          | 3      | AdamW       | Cosine    | 90.33%   |
| 3e-5          | 8          | 5      | AdamW       | Linear    | 90.33%   |
| 3e-5          | 8          | 5      | AdamW       | Cosine    | 89.33%   |
| 3e-5          | 16         | 3      | AdamW       | Linear    | 88.00%   |
| 3e-5          | 16         | 3      | AdamW       | Cosine    | 87.33%   |
| 3e-5          | 16         | 5      | AdamW       | Linear    | 91.00%   |
| 3e-5          | 16         | 5      | AdamW       | Cosine    | 90.00%   |
| 3e-5          | 32         | 3      | AdamW       | Linear    | 52.00%   |
| 3e-5          | 32         | 3      | AdamW       | Cosine    | 73.67%   |
| 3e-5          | 32         | 5      | AdamW       | Linear    | 89.00%   |
| 3e-5          | 32         | 5      | AdamW       | Cosine    | 82.67%   |
| 1e-5          | 8          | 3      | AdamW       | Linear    | 56.67%   |
| 1e-5          | 8          | 3      | AdamW       | Cosine    | 75.67%   |
| 1e-5          | 8          | 5      | AdamW       | Linear    | 89.67%   |
| 1e-5          | 8          | 5      | AdamW       | Cosine    | 87.00%   |
| 1e-5          | 16         | 3      | AdamW       | Linear    | 52.00%   |
| 1e-5          | 16         | 3      | AdamW       | Cosine    | 51.33%   |
| 1e-5          | 16         | 5      | AdamW       | Linear    | 79.33%   |
| 1e-5          | 16         | 5      | AdamW       | Cosine    | 66.00%   |
| 1e-5          | 32         | 3      | AdamW       | Linear    | 51.33%   |
| 1e-5          | 32         | 3      | AdamW       | Cosine    | 51.33%   |
| 1e-5          | 32         | 5      | AdamW       | Linear    | 51.33%   |
| 1e-5          | 32         | 5      | AdamW       | Cosine    | 51.33%   |

### Ανάλυση και Σχολιασμός

1. **Learning Rate (LR) 5e-5:**
    - **Batch Size 8:**
        - The accuracy is high across different epochs and schedulers, with a slight variation between Linear and Cosine schedulers.
    - **Batch Size 16:**
        - Similar performance to batch size 8, with very high accuracy, especially with 5 epochs and Cosine scheduler.
    - **Batch Size 32:**
        - Lower accuracy compared to smaller batch sizes, especially with 3 epochs, but improves significantly with 5 epochs.

2. **Learning Rate (LR) 3e-5:**
    - **Batch Size 8:**
        - Consistently high accuracy, similar to LR 5e-5, with slight variations between schedulers.
    - **Batch Size 16:**
        - Good performance, but slightly lower accuracy compared to batch size 8.
    - **Batch Size 32:**
        - Noticeable drop in accuracy with 3 epochs, but improves with 5 epochs, though not as high as smaller batch sizes.

3. **Learning Rate (LR) 1e-5:**
    - **Batch Size 8:**
        - Moderate accuracy with 3 epochs, significant improvement with 5 epochs.
    - **Batch Size 16:**
        - Very low accuracy with 3 epochs, improves with 5 epochs but still lower compared to higher learning rates.
    - **Batch Size 32:**
        - Consistently low accuracy regardless of epochs and scheduler, suggesting this combination is not effective.

### General Observations:
1. **Effect of Learning Rate:**
    - A higher learning rate (5e-5) tends to yield better accuracy compared to lower learning rates (3e-5 and 1e-5).
    - The lowest learning rate (1e-5) performs poorly, especially with larger batch sizes.

2. **Effect of Batch Size:**
    - Smaller batch sizes (8 and 16) generally perform better, yielding higher accuracy.
    - Larger batch size (32) tends to have lower accuracy, particularly with the lowest learning rate.

3. **Effect of Epochs:**
    - Increasing the number of epochs from 3 to 5 generally improves accuracy, especially noticeable in larger batch sizes.

4. **Effect of Optimizer Scheduler:**
    - Both Linear and Cosine schedulers perform similarly with slight variations.
    - The choice of scheduler does not significantly impact accuracy, though Cosine slightly outperforms Linear in some cases.


- For optimal performance, using a learning rate of 5e-5 with a batch size of 8 or 16, and training for 5 epochs is recommended.
- Learning rate of 1e-5 with large batch sizes, as it consistently yields poor results.


In [ ]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#!pip install transformers[torch]

In [ ]:
#insert code here
from transformers import TrainingArguments, Trainer
from transformers import AdamW, Adafactor, get_scheduler

#args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=16,num_train_epochs=3)

# Experiment with different hyperparameters
learning_rates = [5e-5, 3e-5, 1e-5]
batch_sizes = [8, 16, 32]
num_epochs = [3, 5]
optimizers = {'AdamW': AdamW}
schedulers = {'linear': 'linear', 'cosine': 'cosine'}

for lr in learning_rates:
    for batch_size in batch_sizes:
        for epochs in num_epochs:
            for opt_name, opt_class in optimizers.items():
                for sched_name, sched_type in schedulers.items():
                    # Initialize the model from scratch
                    model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

                    # Set up training arguments
                    training_args = TrainingArguments(
                        output_dir="test_trainer",
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        num_train_epochs=epochs,
                        weight_decay=0.01,
                        learning_rate=lr,
                    )

                    # Initialize the optimizer
                    optimizer = opt_class(model.parameters(), lr=lr)

                    # Initialize the scheduler
                    num_training_steps = len(tokenized_train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs
                    scheduler = get_scheduler(
                        name=sched_type, optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
                    )

                    trainer = Trainer(
                        model=model,
                        args=training_args,
                        train_dataset=tokenized_train_dataset,
                        eval_dataset=tokenized_test_dataset,
                        compute_metrics=compute_metrics,
                        optimizers=(optimizer, scheduler),
                    )

                    # Train the model
                    trainer.train()

                    # Evaluate and store the results
                    eval_result = trainer.evaluate()
                    accuracy = eval_result['eval_accuracy']
                    print(f"LR: {lr}, Batch Size: {batch_size}, Epochs: {epochs}, Optimizer: {opt_name}, Scheduler: {sched_name}, Accuracy: {accuracy}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.278546,0.896667
2,No log,0.372428,0.890000
3,No log,0.371581,0.900000


LR: 5e-05, Batch Size: 8, Epochs: 3, Optimizer: AdamW, Scheduler: linear, Accuracy: 0.9


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.272357,0.893333
2,No log,0.338417,0.893333
3,No log,0.357515,0.896667


...

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.688219,0.513333
2,No log,0.684362,0.513333
3,No log,0.671200,0.513333
4,No log,0.660113,0.513333
5,No log,0.658657,0.513333


LR: 1e-05, Batch Size: 32, Epochs: 5, Optimizer: AdamW, Scheduler: linear, Accuracy: 0.5133333333333333


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.689744,0.513333
2,No log,0.690798,0.513333
3,No log,0.681125,0.513333
4,No log,0.675799,0.513333
5,No log,0.675526,0.513333


LR: 1e-05, Batch Size: 32, Epochs: 5, Optimizer: AdamW, Scheduler: cosine, Accuracy: 0.5133333333333333


In [ ]:
"""
args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=16,num_train_epochs=5)

# insert your code here
# optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)


# scheduler
# Initialize the scheduler
num_training_steps = len(tokenized_train_dataset) // args.per_device_train_batch_size * args.num_train_epochs
scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)
# etc
"""

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
"""
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,

)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)
"""

Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
#1
trained_model=trainer.train()
#save the trained_model and train a from scratch the original
        #  already done above

In [ ]:
#2
trained_model=trainer.train()

In [ ]:
#3
trained_model=trainer.train()

NameError: name 'trainer' is not defined

# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή γνώση (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [4]:
from datasets import load_dataset
from transformers import pipeline
import random
from transformers import AutoModelForMultipleChoice, AutoTokenizer, pipeline

In [5]:
# insert your code here (load dataset)
piqa = load_dataset('piqa')

# Keep just 100 samples
subset_size = 100
piqa_subset = piqa['validation'].shuffle(seed=42).select(range(subset_size))

# Εμφάνιση ενός δείγματος
print(piqa_subset[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for piqa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/piqa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


{'goal': 'How to make Strawberry Kiwi Sauce at home.', 'sol1': 'Boil 1 cup Kiwi (chopped), 1 cup chopped strawberries  with 3/4 cup water and  1 cup Olive pits for 30 min., stirring to keep from scorching over med. heat on the stove top.', 'sol2': 'Boil 1 cup Kiwi (chopped), 1 cup chopped strawberries  with 3/4 cup water and  1 cup sugar for 30 min., stirring to keep from scorching over med. heat on the stove top.', 'label': 1}


Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

### (I wrote that)

Στο παρακάτω κελί χρησιμοποιήσα 2 μοντέλα
  
    "sileod/deberta-v3-small-tasksource-nli",
    "sileod/deberta-v3-base-tasksource-nli"

τα οποία δεν απαιτούν χρήση Tokenizer και είναι fine-tuned στο piqa dataset




In [65]:
# insert your code here (models)

from transformers import AutoTokenizer, AutoModelForMultipleChoice, pipeline


# sileod/deberta-v3-small-tasksource-nli sileod/deberta-v3-base-tasksource-nli
models =[
    "sileod/deberta-v3-small-tasksource-nli",
    "sileod/deberta-v3-base-tasksource-nli",
]
classifiers = {}
for model_name in models:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    classifier = pipeline("zero-shot-classification", model=model_name, tokenizer=tokenizer)
    classifiers[model_name] = classifier

corrects = 0
falses = 0
for model_name, classifier in classifiers.items():
    print("Evaluating model:", model_name)
    corrects = 0
    falses = 0
    # Iterate through each sample in the dataset
    for i in range(0,100):
        text = piqa_subset[i]['goal']
        candidate_labels = [piqa_subset[i]['sol1'], piqa_subset[i]['sol2']]
        # Perform zero-shot classification
        result = classifier(text, candidate_labels)
        prediction = result['labels'][0]
        # Get the ground truth label
        if (prediction == piqa_subset[i]['sol1']) and piqa_subset[i]['label'] == 0:
            corrects += 1
        elif (prediction == piqa_subset[i]['sol2']) and piqa_subset[i]['label'] == 1:
            corrects += 1
        else:
            falses += 1
    print(model_name)
    print("Corrects:", corrects)
    print("Falses:", falses)
    print("Accuracy:", corrects/(falses+corrects))


tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Evaluating model: sileod/deberta-v3-small-tasksource-nli


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


sileod/deberta-v3-small-tasksource-nli
Corrects: 63
Falses: 37
Accuracy: 0.63
Evaluating model: sileod/deberta-v3-base-tasksource-nli
sileod/deberta-v3-base-tasksource-nli
Corrects: 74
Falses: 26
Accuracy: 0.74


In [2]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [6]:
# Preprocess function
def preprocess(example):
    example['first_sentence'] = [example['goal']] * 2
    example['second_sentence'] = [example['sol1'], example['sol2']]
    return example

# Apply preprocessing
piqa_subset_preprocessed = piqa_subset.map(preprocess)
print(piqa_subset_preprocessed)

Dataset({
    features: ['goal', 'sol1', 'sol2', 'label', 'first_sentence', 'second_sentence'],
    num_rows: 100
})


In [141]:
# Function to tokenize input


In [24]:
from transformers import AutoModelForMultipleChoice
from transformers import AutoTokenizer

# Load the pre-trained model


model1 = AutoModelForMultipleChoice.from_pretrained('bert-base-uncased')
tokenizer1 = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function1(examples):
    tokenized_examples=tokenizer1(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples
###-----------------------------------------------------------------------------
model2 = AutoModelForMultipleChoice.from_pretrained('roberta-base')
tokenizer2 = AutoTokenizer.from_pretrained('roberta-base')

def tokenize_function2(examples):
    tokenized_examples=tokenizer2(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples
##----------------------------------------------------------------------------------
model3 = AutoModelForMultipleChoice.from_pretrained('albert-base-v2')
tokenizer3 = AutoTokenizer.from_pretrained('albert-base-v2')

def tokenize_function3(examples):
    tokenized_examples=tokenizer3(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples
#____________________________________________________________________________________
model4 = AutoModelForMultipleChoice.from_pretrained('sledz08/finetuned-bert-piqa')
tokenizer4 = AutoTokenizer.from_pretrained('sledz08/finetuned-bert-piqa')

def tokenize_function4(examples):
    tokenized_examples=tokenizer4(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples

#_______________________________________________________________________________________________________________________________
model5 = AutoModelForMultipleChoice.from_pretrained('google/electra-base-discriminator')
tokenizer5 = AutoTokenizer.from_pretrained('google/electra-base-discriminator')

def tokenize_function5(examples):
    tokenized_examples=tokenizer5(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples


Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of AlbertForMultipleChoice were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of ElectraForMultipleChoice were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [1]:
!pip uninstall transformers accelerate -y
!pip install transformers[torch] accelerate -U

Found existing installation: transformers 4.41.0
Uninstalling transformers-4.41.0:
  Successfully uninstalled transformers-4.41.0
Found existing installation: accelerate 0.30.1
Uninstalling accelerate-0.30.1:
  Successfully uninstalled accelerate-0.30.1
  Using cached transformers-4.41.0-py3-none-any.whl (9.1 MB)
  Using cached accelerate-0.30.1-py3-none-any.whl (302 kB)


In [14]:
from transformers import TrainingArguments, Trainer

tokenized_test_dataset1 = piqa_subset_preprocessed.map(tokenize_function1)
trainer = Trainer(
      model= model1,
      eval_dataset=tokenized_test_dataset1,
      compute_metrics=compute_metrics,
    )
trainer.evaluate()

{'eval_loss': 0.6933594346046448,
 'eval_accuracy': 0.43,
 'eval_runtime': 3.052,
 'eval_samples_per_second': 32.766,
 'eval_steps_per_second': 4.26}

In [15]:
from transformers import TrainingArguments, Trainer

tokenized_test_dataset2 = piqa_subset_preprocessed.map(tokenize_function2)
trainer = Trainer(
      model= model2,
      eval_dataset=tokenized_test_dataset2,
      compute_metrics=compute_metrics,
    )
trainer.evaluate()

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'eval_loss': 0.6932838559150696,
 'eval_accuracy': 0.46,
 'eval_runtime': 3.5746,
 'eval_samples_per_second': 27.976,
 'eval_steps_per_second': 3.637}

In [16]:
from transformers import TrainingArguments, Trainer

tokenized_test_dataset3 = piqa_subset_preprocessed.map(tokenize_function3)
trainer = Trainer(
      model= model3,
      eval_dataset=tokenized_test_dataset3,
      compute_metrics=compute_metrics,
    )
trainer.evaluate()

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'eval_loss': 0.6937612295150757,
 'eval_accuracy': 0.48,
 'eval_runtime': 4.768,
 'eval_samples_per_second': 20.973,
 'eval_steps_per_second': 2.727}

In [18]:
from transformers import TrainingArguments, Trainer

tokenized_test_dataset4 = piqa_subset_preprocessed.map(tokenize_function4)
trainer = Trainer(
      model= model4,
      eval_dataset=tokenized_test_dataset4,
      compute_metrics=compute_metrics,
    )
trainer.evaluate()

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'eval_loss': 0.7027284502983093,
 'eval_accuracy': 0.66,
 'eval_runtime': 3.1088,
 'eval_samples_per_second': 32.167,
 'eval_steps_per_second': 4.182}

In [25]:
from transformers import TrainingArguments, Trainer

tokenized_test_dataset5 = piqa_subset_preprocessed.map(tokenize_function5)
trainer = Trainer(
      model= model5,
      eval_dataset=tokenized_test_dataset5,
      compute_metrics=compute_metrics,
    )
trainer.evaluate()

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'eval_loss': 0.6937179565429688,
 'eval_accuracy': 0.44,
 'eval_runtime': 3.906,
 'eval_samples_per_second': 25.602,
 'eval_steps_per_second': 3.328}

## B2. Truthful QA

### Sentence Transformers

Οι sentence transformers χρησιμοποιούνται για να δημιουργήσουν embeddings προτάσεων, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [59]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [67]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

0.40488467

Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή.

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


In [48]:
# Import the necessary library
from datasets import load_dataset

# Load the dataset
dataset_qa = load_dataset('truthful_qa', 'generation')

# Shuffle the dataset with a fixed seed for reproducibility

dataset_filtered = dataset_qa['validation'].filter(lambda e: len(e['correct_answers'])>=2).shuffle().select(range(100))

dataset_filtered_labeled = dataset_filtered.add_column('label', [0]*100)

print(dataset_filtered_labeled)


Flattening the indices:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['type', 'category', 'question', 'best_answer', 'correct_answers', 'incorrect_answers', 'source', 'label'],
    num_rows: 100
})


In [30]:
!pip install -U sentence-transformers

In [49]:
# Preprocess function
def preprocess2(example):
    example['first_sentence']=[example['question']]*4
    example['second_sentence']=[example['best_answer'],example['correct_answers'][0],example['correct_answers'][1],example['incorrect_answers'][0]]
    return example


# Apply preprocessing
dataset_filtered_with_label_processed = dataset_filtered_labeled.map(preprocess2)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [55]:
# insert your code here (load models for semantic similarity and QA)
from sentence_transformers import SentenceTransformer

smodel1 = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
smodel2 = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
smodel3 = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')
smodel4 = SentenceTransformer('sentence-transformers/distilbert-base-nli-stsb-mean-tokens')
smodel5 = SentenceTransformer('sentence-transformers/stsb-roberta-base')
smodel6 = SentenceTransformer('sentence-transformers/paraphrase-distilroberta-base-v2')

###-------------------------------------------------------------------------------------------
model1 = AutoModelForMultipleChoice.from_pretrained('bert-base-uncased')
tokenizer1 = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function1(examples):
    tokenized_examples=tokenizer1(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples
###-----------------------------------------------------------------------------
model2 = AutoModelForMultipleChoice.from_pretrained('roberta-base')
tokenizer2 = AutoTokenizer.from_pretrained('roberta-base')

def tokenize_function2(examples):
    tokenized_examples=tokenizer2(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples
##----------------------------------------------------------------------------------
model3 = AutoModelForMultipleChoice.from_pretrained('albert-base-v2')
tokenizer3 = AutoTokenizer.from_pretrained('albert-base-v2')

def tokenize_function3(examples):
    tokenized_examples=tokenizer3(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples
#____________________________________________________________________________________
model4 = AutoModelForMultipleChoice.from_pretrained('sledz08/finetuned-bert-piqa')
tokenizer4 = AutoTokenizer.from_pretrained('sledz08/finetuned-bert-piqa')

def tokenize_function4(examples):
    tokenized_examples=tokenizer4(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples

#_______________________________________________________________________________________________________________________________
model5 = AutoModelForMultipleChoice.from_pretrained('google/electra-base-discriminator')
tokenizer5 = AutoTokenizer.from_pretrained('google/electra-base-discriminator')

def tokenize_function5(examples):
    tokenized_examples=tokenizer5(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples





/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of AlbertForMultipleChoice were not initialized from the model checkpoint a

In [52]:

# insert your code here (function for optimal correct answers & semantic similarity)
def optimal_predictions(predictions,model):
    for i in range(len(predictions)):
      if (predictions[i]==0):
        predictions[i]=0
      elif (predictions[i]==3):
        predictions[i]=1
      else :
        sentences = [dataset_filtered_labeled[i]['correct_answers'][predictions[i]-1], dataset_filtered_labeled[i]['best_answer']]
        embeddings = model.encode(sentences)
        if (get_cosine_similarity(embeddings[0], embeddings[1])>0.95):
          predictions[i]=0
        else:
          predictions[i]=1
    return predictions



In [53]:
import numpy as np
import evaluate
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics2(eval_pred, sentence_model):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    optpredictions = optimal_predictions(predictions, sentence_model)
    return metric.compute(predictions=optpredictions, references=labels)
# We define a second accuracy computing function for calculating the percentage of optimal correct answers

In [57]:
# Define a list of models and datasets

tokenized1_val_100 = dataset_filtered_with_label_processed.map(tokenize_function1)
tokenized2_val_100 = dataset_filtered_with_label_processed.map(tokenize_function2)
tokenized3_val_100 = dataset_filtered_with_label_processed.map(tokenize_function3)
tokenized4_val_100 = dataset_filtered_with_label_processed.map(tokenize_function4)
tokenized5_val_100 = dataset_filtered_with_label_processed.map(tokenize_function5)

models_and_datasets = [
    (model1, tokenized1_val_100, "bert-base-uncased"),
    (model2, tokenized2_val_100, "distilbert-base-uncased"),
    (model3, tokenized3_val_100, "roberta-base"),
    (model4, tokenized4_val_100, "albert-base-v2"),
    (model5, tokenized5_val_100, "google/electra-base-discriminator"),
]


# Sentence Transformer models
sentence_models = [
    ("sentence-transformers/all-MiniLM-L6-v2", smodel1),
    ("sentence-transformers/all-MiniLM-L12-v2", smodel2),
    ("sentence-transformers/multi-qa-MiniLM-L6-cos-v1", smodel3),
    ("sentence-transformers/distilbert-base-nli-stsb-mean-tokens", smodel4),
    ("sentence-transformers/stsb-roberta-base", smodel5),
    ("sentence-transformers/paraphrase-distilroberta-base-v2", smodel6)
]



Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [60]:
results = []

# Evaluate the multiple-choice models with sentence transformers
for model, tokenized_val_dataset, model_name in models_and_datasets:
    for sentence_model_name, sentence_model in sentence_models:
        # Initialize Trainer
        trainer = Trainer(
            model=model,
            eval_dataset=tokenized_val_dataset,
            compute_metrics=lambda eval_pred: compute_metrics2(eval_pred, sentence_model),
        )

        # Evaluate the model
        eval_result = trainer.evaluate()
        accuracy = eval_result["eval_accuracy"]

        # Store the results
        results.append((model_name, sentence_model_name, accuracy))

In [ ]:
# Print results as a markdown table using tabulate
from tabulate import tabulate
headers = ["Model", "Sentence Model", "Accuracy"]
table = tabulate(results, headers=headers, tablefmt="github")
print(table)

## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [54]:
from datasets import load_dataset

from datasets import load_dataset
from transformers import pipeline
import numpy as np

# Load the Winogrande dataset
dataset = load_dataset('allenai/winogrande', 'winogrande_debiased')

# Shuffle and select 100 samples from the train subset
dataset = dataset['train'].shuffle(seed=42).select(range(100))



/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for allenai/winogrande contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/allenai/winogrande
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/9248 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1767 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1267 [00:00<?, ? examples/s]

Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

In [89]:
# insert your code here (load models)


# Transform the dataset for binary classification
def prepare_examples(example):
    sentence = example['sentence']
    option1 = example['option1']
    option2 = example['option2']
    question = sentence.replace("_", "[MASK]")
    example['input1'] = question.replace("[MASK]", option1)
    example['input2'] = question.replace("[MASK]", option2)
    return example

dataset_prepared = dataset.map(prepare_examples)

# Define a function to compute accuracy
def compute_accuracy(predictions, labels):
    return np.mean(np.array(predictions) == np.array(labels))

# Define models to evaluate
model_names = [
    "distilbert-base-uncased-finetuned-sst-2-english",
    "roberta-large-mnli",
    "bert-base-uncased"
]




/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for allenai/winogrande contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/allenai/winogrande
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [90]:
# insert your code here (create pipelines)

# Create pipelines for each model
pipelines = {name: pipeline("text-classification", model=name) for name in model_names}


config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [92]:
# insert your code here (function for predicting best fill)
# Evaluate models
accuracies = {}

for model_name, clf_pipeline in pipelines.items():
    predictions = []
    for example in dataset:
        input1 = example['option1']
        input2 = example['option2']

        # Get scores for both options
        result1 = clf_pipeline(input1)[0]
        result2 = clf_pipeline(input2)[0]

        # Predict the option with the higher score
        if result1['score'] > result2['score']:
            predictions.append(1)
        else:
            predictions.append(2)

    # Compare with the actual labels
    actual_labels = [1 if answer == '1' else 2 for answer in dataset['answer']]
    accuracy = compute_accuracy(predictions, actual_labels)
    accuracies[model_name] = accuracy

# Print the accuracies of the models
for model_name, accuracy in accuracies.items():
    print(f"Accuracy of {model_name}: {accuracy:.2f}")


Accuracy of distilbert-base-uncased-finetuned-sst-2-english: 0.46
Accuracy of roberta-large-mnli: 0.51
Accuracy of bert-base-uncased: 0.48
